In [0]:
!pip install yfinance
!pip install numpy-financial

In [0]:
import yfinance as yf
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import datetime, timedelta, date
from pandas.tseries.offsets import BDay
import numpy as np
import numpy_financial as npf
from typing import List, Dict, Optional, Tuple
from collections import defaultdict

In [0]:
df = yf.download('SCHD')
df = df.reset_index(drop=False)
display(df)

In [0]:
def calculate_annualized_return(invest_dates: List[date], invest_values: List[int], final_portfolio_value: float) -> float:
    """
    Calculate the annualized return

    Args:
        invest_dates: a list of investment dates
        invest_values: a list of investment in dollars
        final_portfolio_value: the portfolio value at the end of investment
    
    Returns:
        annualized_return rate
    """
    # get annualized contributions
    annual_contri_d = defaultdict(int)
    for invest_date, invest_value in zip(invest_dates, invest_values):
        year = invest_date.year
        annual_contri_d[year] += invest_value
        
    annual_contri_list = []
    for invest_year in annual_contri_d.keys():
        annual_contri_list.append(annual_contri_d[invest_year])

    # call irr to calculate return
    a = np.array([-1 * invest for invest in annual_contri_list] + [final_portfolio_value])

    return npf.irr(a)

def calculate_sharpe_ratio(invest_dates: List[date], invest_values: List[int], buy_in_shares: List[int], stock_data : pd.DataFrame, risk_free_rate: Optional[float] = 0.02) -> float:
    """
    Calculate annualized sharpe ratio. Only consider keeping buy-in without any sell.
    A year's return rate = (year_end_value - last_year_end_value - investment_in_year) / last_year_end_value

    Args:
        invest_dates: a list of investment dates
        invest_values: a list of investment in dollars
        buy_in_shares: 
        final_portfolio_value: the portfolio value at the end of investment
    
    """
    # calculate each year's end value and total investment in a year
    yearly_values = {}
    yearly_investments = {}

    df = pd.DataFrame({
        'date': invest_dates, 
        'value': invest_values, 
        'buy_in_share': buy_in_shares
    })
    df = df.set_index('date')
    df['hold_share'] = df['buy_in_share'].cumsum()
    min_year = invest_dates[0].year
    max_year = invest_dates[-1].year
    for year in range(min_year, max_year): # intentionally skip last year because it may not be complete yet
        year_df = df[str(year)]
        year_end_share = year_df.iloc[-1]['hold_share']
        year_end_price = stock_data[str(year)].iloc[-1]['Close']
        year_end_value = year_end_share * year_end_price
        yearly_values[year] = year_end_value
        yearly_invest = year_df['value'].sum()
        yearly_investments[year] = yearly_invest
    
    # calcualate yearly return
    yearly_return_rate = []
    for year in range(min_year+1, max_year):
        last_year_end_value = yearly_values[year-1]
        this_year_end_value = yearly_values[year]
        this_year_invest = yearly_investments[year]
        return_rate = (this_year_end_value - last_year_end_value - this_year_invest) / last_year_end_value
        yearly_return_rate.append(return_rate)

    # calculate sharpe ratio
    yearly_return_rate = np.array(yearly_return_rate)
    excess_returns = yearly_return_rate - risk_free_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(yearly_return_rate)
    
    return sharpe_ratio

In [0]:
class Trade:
    ETF_DF = pd.DataFrame(
        [("VGT", '2004-01-26' , 0.10, 'Technology Sector'), ("IVV",'2000-05-15', 0.03, 'Large-Cap Blend'), 
               ("SWPPX", '1997-05-19', 0.02, 'Large-Cap Blend'), ("VTI",'2001-05-24' , 0.03, 'Large Blend'), 
               ("DIA",'1998-01-13', 0.16, 'Large Value'), ("QQQ",'1999-03-10', 0.20, 'Large Growth'), ("IWM",'2000-05-22', 0.19, 'Small-Cap Blend'), 
               ("DBC",'2006-02-03', 0.85, 'Commodities Broad Basket'), ("SWTSX",'1999-06-01', 0.03, 'Large-Cap Blend'), 
               ("WFIVX",'1999-02-01', 0.54, 'Large-Cap Growth'), ("VNQ",'2004-09-23', 0.12, 'Real Estate'), ("SMH",'2011-12-20', 0.35, 'Semiconductors'), 
               ("FNCMX", '2003-09-25', 0.29, 'Large-Cap Growth'), ("OEF", '2000-10-23', 0.20, 'Large Blend'), 
               ("MDY", '1995-05-04', 0.23, 'Mid-Cap Blend'), ("IWO",'2000-07-24', 0.23, 'Small Growth'), ("VUG", '2004-01-26', 0.04, 'Large-Cap Growth'), 
               ("NASDX", '2000-01-18', 0.50, 'Large Growth'), ("SDY", '2005-11-08', 0.35, 'Mid-Cap Value'), 
               ("SSO", '2006-06-19', 0.89, 'Leveraged Large-Cap Blend'), ("GLD", '2004-11-18', 0.40, 'Gold') , 
               ("XLK", '1998-12-16', 0.10, 'Technology Sector'), ("XLV", '1998-12-16', 0.10, 'Healthcare Sector'), 
               ("XLE", '1998-12-16', 0.10, 'Energy Sector'), ("IWV", '2000-05-22', 0.20, 'Large Blend'), ("VGELX", '2001-11-12', 0.25, 'Large-Cap Value'),
               ("VINIX", '1990-07-31', 0.02, 'Large Blend'), ("AGTHX", '1973-12-01', 1.39, 'Large-Cap Growth'), 
               ("VWO", '2005-03-04', 0.08, 'Emerging Markets'), ("VXF", '2001-12-27', 0.06, 'Mid-Cap Blend'), ("SCHD", '2011-10-20', 0.06, 'Large Value'),
               ("VOO",'2010-09-07' ,0.14, 'Large-Cap Blend'),  ('VIG','2006-04-21' , 0.08, 'Large Blend'), ('IWR', '2001-07-17', 0.18, 'Mid-Cap Blend'),
               ('VOE', '2006-08-17', 0.07, 'Mid-Cap Value'), ('VB', '2004-01-26', 0.03, 'Small Blend'), ('VO', '2001-11-12', 0.05, 'Mid-Cap Blend'),
               ('SCHA', '2009-11-03' ,0.04, 'Small Blend'), ('VHT', '2004-02-05', 0.10, 'Healthcare Sector'), 
               ('IHF', '2006-05-01', 0.39, 'Healthcare Sector')],
        columns=['ticker', 'inception_date', 'expense_ratio', 'category']
    )

    def __init__(self) -> None:
        self.hist_df = self.__load_data()

    def __load_data(self) -> pd.DataFrame:
        """
        Return a df with Date, Ticker, Open, Low, High, Close (all prices are adjusted)
        """
        dfs = []
        etfs = self.ETF_DF['ticker'].values
        for etf in etfs:
            df = yf.download(etf)
            df = df.reset_index(drop=False)
            df['Adj'] = df['Adj Close'] - df['Close']
            df['Adj Open'] = df['Open'] + df['Adj']
            df['Adj Low'] = df['Low'] + df['Adj']
            df['Adj High'] = df['High'] + df['Adj']
            df['Date'] = df['Date'].apply(lambda x: x.date())
            df['Ticker'] = etf
            df = df[['Date', 'Ticker', 'Adj Open', 'Adj Low', 'Adj High', 'Adj Close']]
            df.columns = ['Date', 'Ticker', 'Open', 'Low', 'High', 'Close']
            dfs.append(df)

        df = pd.concat(dfs, ignore_index=True)
        return df
    
    def __get_trade_period(self, etfs: List[str], start_dt: Optional[date] = None) -> Tuple[date, date]:
        """
        Return the start_dt and end_dt of the trade.
        """
        if start_dt is None:
            start_dts = [self.hist_df.loc[self.hist_df['Ticker'] == etf]['Date'].min() for etf in etfs]
            start_dt = max(start_dts)

        end_dts = [self.hist_df.loc[self.hist_df['Ticker'] == etf]['Date'].max() for etf in etfs]
        end_dt = min(end_dts)

        return start_dt, end_dt
    
    def __get_trade_freq(self, freq: str, trading_day: str) -> str:
        """
        freq: weekly/biweekly/monthly
        trading_day: MON/TUE/WED/THU/FRI/START/END
        """
        if freq == 'weekly':
            freq_s = 'W-' + trading_day
        elif freq == 'biweekly':
            freq_s = 'SM'
            if trading_day == 'START':
                freq_s += 'S'
        else:
            freq_s = 'BM'
            if trading_day == 'START':
                freq_s += 'S'
        
        return freq_s
    
    def __get_trade_days(
        self,  
        etfs: List[str], 
        start_dt: Optional[date],
        freq: str, 
        trading_day: str
    ) -> np.ndarray:
        """
        Get a an array of trading days

        Returns:
            an np array of dates
        """
        # get start and end_dt
        start_dt, end_dt = self.__get_trade_period(etfs, start_dt)

        # get freq
        if freq == None:
            freq_s = 'B'
        else:
            freq_s = self.__get_trade_freq(freq, trading_day)

        # generate dates
        dates = pd.date_range(start=start_dt, end=end_dt, freq=freq_s, inclusive='both').date
        # find dates that are in the index of hist_df
        all_trade_dates = self.hist_df['Date'].values
        trade_dates = []
        for day in dates:
            v = day
            while v not in all_trade_dates:
                v = v + BDay(1)
            trade_dates.append(v)
        trade_dates = np.array(trade_dates)
        return trade_dates
    
    def __get_day_price(
        self,
        day: date,
        etfs: List[str],
        metric: str
    ) -> List[float]:
        """
        Get the price of the etfs on a certain day

        Args:
            day: date
            etfs: Tickers
            metric: Open/Low/High/Close
        """
        price_df = self.hist_df.loc[(self.hist_df['Date'] == day) & (self.hist_df['Ticker'].isin(etfs))][['Ticker', metric]]
        price_df = price_df.set_index('Ticker')
        price_s = price_df[metric]
        price_s = price_s[etfs]
        return price_s.values.tolist()
    
    def __calculate_total_value(
        self,
        day: date,
        invest_df: pd.DataFrame
    ) -> pd.Series:
        """
        Calculate the total value of all the holdings based on the open price

        Args:
            day: 
            invest_df: a pd.DataFrame with Date, etfs and Invest_Value that lists the history of transactions

        Returns:
            a pd.Series that shows the values of each ETF, indexed by ETF tickers
        """
        # calculate total holding df
        holding_df = invest_df.drop(columns=['Date', 'Invest_Value'])
        holding_df = pd.DataFrame([holding_df.sum()])
        etfs = holding_df.columns
        holding_df = holding_df.melt(value_vars=etfs, var_name='Ticker', value_name='Holding')

        # join 
        price_df = self.hist_df.loc[(self.hist_df['Date'] == day) & (self.hist_df['Ticker'].isin(etfs))]
        value_df = holding_df.merge(price_df, on='Ticker', how='inner')

        # calculate each etf's total value
        value_df['Value'] = value_df['Open'] * value_df['Holding']
        value_s = pd.Series(value_df['Value'].values, index=value_df['Ticker'].values)
        value_s = value_s[etfs]
        return value_s

    def __calculate_annual_contribution(
        self,
        invest_df: pd.DataFrame
    ) -> np.array:
        """
        Calculate the annual investment contribution value

        Args:
            invest_df: a pd.DataFrame with Date, etfs and Invest_Value that lists the history of transactions

        Returns:
            an np array of annual investment
        """
        invest_df = invest_df.copy()
        invest_df['year'] = invest_df['Date'].apply(lambda x: x.year)
        annual_contri = (
            invest_df
                .groupby('year')['Invest_Value']
                .sum()
                .reset_index(drop=False)
                .sort_values(by='year')
                ['Invest_Value'].values
        )
        return annual_contri
    
    def __calculate_annualized_return(
        self,
        invest_df: pd.DataFrame
    ) -> float:
        """
        Calculate annualized return based on the investment history and the lastest ETF price

        Args:
            invest_df: a pd.DataFrame with Date, etfs and Invest_Value that lists the history of transactions
        
        Returns:
            annualized return
        """
        # calculate final_portfolio_value
        last_day = self.hist_df['Date'].max()
        final_portfolio_value = self.__calculate_total_value(last_day, invest_df).sum()

        # get annual contri
        annual_contri = self.__calculate_annual_contribution(invest_df)

        # call irr to calculate return
        a = np.append((-1) * annual_contri, [final_portfolio_value])

        return npf.irr(a)
    
    def __calculate_year_end_value(self, invest_df: pd.DataFrame) -> np.array:
        """
        Calculate the year-end value of the portfolio

        Args:
            invest_df: a pd.DataFrame with Date, etfs and Invest_Value that lists the history of transactions

        Returns:
            an np array of year-end value
        """
        # get all years
        first_year = invest_df['Date'].values[0].year
        last_year = invest_df['Date'].values[-1].year

        # for each year
        year_end_values = []
        for year in range(first_year, last_year):
            # find last day
            last_day = date(year, 12, 31)
            # take a sub_df from invest_df
            sub_df = invest_df.loc[invest_df['Date'] <= last_day]
            last_day = sub_df.iloc[-1]['Date']
            # calculate value
            value = self.__calculate_total_value(last_day, sub_df).sum()
            year_end_values.append(value)

        return np.array(year_end_values)
    
    def __calculate_sharpe_ratio(
        self,
        invest_df: pd.DataFrame,
        risk_free_rate: float = 0.02
    ) -> float:
        """
        Calculate sharpe ratio based on the investment history and the lastest ETF price

        Args:
            invest_df: a pd.DataFrame with Date, etfs and Invest_Value that lists the history of transactions
        
        Returns:
            sharpe ratio
        """
        # calculate annual contribution
        annual_contris = self.__calculate_annual_contribution(invest_df)

        # calculate annual year-end value
        annual_values = self.__calculate_year_end_value(invest_df)

        # calculate annual return
        annual_rates = []
        for prev_annual_value, next_annual_contri, next_annual_value in zip(annual_values[:-2], annual_contris[1:-1], annual_values[1:-1]):
            annual_rate = (next_annual_value - next_annual_contri - prev_annual_value) / prev_annual_value
            annual_rates.append(annual_rate)

        # calculate sharpe ratio
        annual_rates = np.array(annual_rates)
        excess_returns = annual_rates - risk_free_rate
        sharpe_ratio = np.mean(excess_returns) / np.std(annual_rates)
        return sharpe_ratio
    
    def __dca_buy(
        self,
        day: date,
        etfs: List[str],
        shares: Optional[List[int]] = [2], 
        amt: Optional[float] = 500,  
        weights: Optional[List[float]] = None
    ) -> pd.Series:
        """
        Use dca to buy in etfs on a certain date

        Args:
            day:
            etfs: tickers
            shares (Optional): number of shares to buy of each etf
            amt (Optional): dollars to invest each time. Only share or amt needs to be specified. If both are provided, share has a higher priority, and amt will be ignored
            weights (Optional): It specifies the portfolio weights across ETFs. Sum to 1 

        Returns:
            a pd.Series that includes Date, etf shares to buy and Invest_Value
        """
        day_df = self.hist_df.loc[(self.hist_df['Date'] == day) & (self.hist_df['Ticker'].isin(etfs))]
        if shares is not None:
            share_df = pd.DataFrame({'shares': shares, 'Ticker': etfs})
            buy_df = share_df.merge(day_df, on='Ticker', how='inner')
            buy_df['Value'] = buy_df['shares'] * buy_df['Open']
            amt = buy_df['Value'].sum()
        else:
            amts = amt * np.array(weights)
            amts_df = pd.DataFrame({
                'Ticker': etfs,
                'Value': amts
            })
            buy_df = amts_df.merge(day_df, on='Ticker', how='inner')
            etfs = buy_df['Ticker'].values.tolist()
            buy_df['shares'] = buy_df['Value'] / buy_df['Open']
            shares = buy_df['shares'].values.tolist()
        
        s = pd.Series(
            [day] + shares + [amt],
            index=['Date'] + etfs + ['Invest_Value']
        )
        return s
    
    def __rebalance(
        self, 
        day: date,
        etfs: List[str],
        weights: List[float],
        invest_df: pd.DataFrame
    ) -> pd.Series:
        """
        Perform rebalance at the date based on Open Price

        Args:
            day: which day to perform rebalance
            etfs: tickers
            weights: sum to 1
            invest_df: a pd.DataFrame with Date, etfs and Invest_Value that lists the history of transactions

        Returns:
            a pd.Series that includes Date, etf shares to buy and Invest_Value. The ETF shares represent the changes in the holdings of all ETFs and the Invest_Value=0
        """
        # calculate total shares of each etf
        prior_shares = invest_df[etfs].sum()

        # calculate total value of each etf
        prior_values = self.__calculate_total_value(day, invest_df)

        # calculate total portfolio value
        prior_total_value = prior_values.sum()

        # calculate the rebalanced value of each etf
        after_values = prior_total_value * np.array(weights)

        # get the day open price of each etf
        prices = self.__get_day_price(day, etfs, 'Open')

        # calculate the rebalancd shares of each etf
        after_shares = after_values / np.array(prices)

        # calculate share changes
        s = after_shares - prior_shares
        s['Date'] = day
        s['Invest_Value'] = 0
        return s

    def dca_trade(
        self,
        etfs: List[str], 
        freq: str, 
        shares: Optional[List[int]] = [2], 
        amt: Optional[float] = 500, 
        trading_day: Optional[str] = 'MON', 
        rebalance_flag: bool = False, 
        weights: Optional[List[float]] = None, 
        start_dt: Optional[date] = None
    ) -> Tuple[float, float]:
        """
        Calculate the annualized return and sharpe ratio of dollar cost average on a single etf
        
        Args:
            etfs: tickers
            freq: weekly/biweekly/monthly
            shares (Optional): number of shares to buy of each etf
            amt (Optional): dollars to invest each time. Only share or amt needs to be specified. If both are provided, share has a higher priority, and amt will be ignored
            trading_day (Optional): MON/TUE/WED/THU/FRI/START/END
            rebalance_flag: bool whether to perform a rebalance at the beginning of a year
            weights (Optional): It specifies the portfolio weights across ETFs. Sum to 1 
            start_dt (Optional): If not provided, will use the inception date

        Returns:
            annualized_return
            sharpe_ratio
        """
        invest_df = pd.DataFrame(columns=['Date'] + etfs + ['Invest_Value'])

        # get trading days
        trade_days = self.__get_trade_days(etfs, start_dt, freq, trading_day)
        last_day_year = trade_days[0].year

        # on each trade day
        for day in trade_days:
            day_year = day.year
            # if a new year and rebalance_flag
            if day_year > last_day_year and rebalance_flag:
                # get the first day of a year and perform rebalance
                first_business_day = self.hist_df.loc[self.hist_df['Date'] >= date(day_year, 1, 1), 'Date'].min()
                invest_s = self.__rebalance(first_business_day, etfs, weights, invest_df)
                invest_df = pd.concat([invest_df, pd.DataFrame([invest_s])], ignore_index=True)
                last_day_year = day_year

            # calculate shares to buy and total invest value
            invest_s = self.__dca_buy(day, etfs, shares, amt, weights)
            # append it to invest_df
            invest_df = pd.concat([invest_df, pd.DataFrame([invest_s])], ignore_index=True)

        # calculate return rate and sharpe ratio
        annualized_return = self.__calculate_annualized_return(invest_df)
        sharpe_ratio = self.__calculate_sharpe_ratio(invest_df)
        return annualized_return, sharpe_ratio


    def rule_trade(
        self,
        etfs: List[str], 
        shares: List[int], 
        thresholds: List[float], 
        rebalance_flag: bool = False, 
        weights: Optional[List[float]] = None,
        start_dt: Optional[date] = None    
    ) -> Tuple[float, float]:
        """
        Calculate annualized return and sharpe ratio of a certain trading strategy

        Args:
            etfs: tickers
            shares (Optional): number of shares to buy of each etf
            thresholds: a list of threshold. Only buy in certain shares if the price change is beyond the thresholds. 
            rebalance_flag: bool whether to perform a rebalance at the beginning of a year
            weights (Optional): It specifies the portfolio weights across ETFs. Sum to 1 
            start_dt (Optional): If not provided, will use the inception date

        Returns:
            annualized_return
            sharpe_ratio
        """
        # get all trade days
        start_dt, end_dt = self.__get_trade_period(etfs, start_dt)
        trade_days = pd.date_range(start_dt, end_dt, freq='B', inclusive='both').date
        trade_days = np.intersect1d(trade_days, self.hist_df['Date'].values)

        # get low and high percentage
        etf_df = self.hist_df.loc[self.hist_df['Ticker'].isin(etfs)]
        etf_df['Low_Pctg'] = etf_df['Low'] / etf_df['Open'] - 1
        etf_df['High_Pctg'] = etf_df['High'] / etf_df['Open'] - 1
        
        invest_df = pd.DataFrame(columns=['Date'] + etfs + ['Invest_Value'])
        last_day_year = trade_days[0].year
        # for each day
        for day in trade_days:
            day_year = day.year
            # if a new year and rebalance_flag
            if day_year > last_day_year and rebalance_flag:
                # get the first day of a year and perform rebalance
                first_business_day = self.hist_df.loc[self.hist_df['Date'] >= date(day_year, 1, 1), 'Date'].min()
                invest_s = self.__rebalance(first_business_day, etfs, weights, invest_df)
                invest_df = pd.concat([invest_df, pd.DataFrame([invest_s])], ignore_index=True)
                last_day_year = day_year

            # invest_value = 0
            invest_value = 0
            # construct invest_s
            invest_s = pd.Series([day], index=['Date'])
            # for each etf
            for etf, threshold, share in zip(etfs, thresholds, shares):
                etf_s = etf_df.loc[(etf_df['Ticker'] == etf) & (etf_df['Date'] == day)].iloc[0]
                # check if change >= threshold
                trade_flag = False
                if threshold > 0:
                    trade_flag = (etf_s['High_Pctg'] >= threshold)
                else:
                    trade_flag = (etf_s['Low_Pctg'] <= threshold)
                # if yes, get share and add invest_value, else 0
                if trade_flag:
                    unit_price = etf_s['Open'] * (1 + threshold)
                    buy_value = unit_price * share
                    invest_value += buy_value
                else:
                    share = 0
                invest_s[etf] = share

            # if invest_value > 0 add invest_s to invest_df
            if invest_value > 0:
                invest_s['Invest_Value'] = invest_value
                invest_df = pd.concat([invest_df, pd.DataFrame([invest_s])], ignore_index=True)
        
        # calculate return rate and sharpe ratio
        annualized_return = self.__calculate_annualized_return(invest_df)
        sharpe_ratio = self.__calculate_sharpe_ratio(invest_df)
        return annualized_return, sharpe_ratio

In [0]:
trade = Trade()
display(trade.hist_df)

#### DCA

In [0]:
etfs_choices = [['VGT'], ['VOO'], ['SCHD'], ['VGT', 'VOO'], ['VGT', 'SCHD'], ['VOO', 'SCHD'], ['VGT', 'VOO', 'SCHD']]
freq_choices = ['weekly', 'biweekly', 'monthly']
share_dict = {
    'VGT': 2,
    'VOO': 2,
    'SCHD': 10
}

In [0]:
ss = []
for etfs in etfs_choices:
    shares = [share_dict[etf] for etf in etfs]
    for freq in freq_choices:
        # config trading_day
        if freq == 'weekly':
            trading_day_choices = ['MON', 'TUE', 'WED', 'THU', 'FRI']
        else:
            trading_day_choices = ['START', 'END']
        for trading_day in trading_day_choices:
            if len(etfs) > 1:
                rebalance_flag_choices = [True, False]
            else:
                rebalance_flag_choices = [False]
            for rebalance_flag in rebalance_flag_choices:
                if rebalance_flag:
                    if len(etfs) == 2:
                        weights_choices = [[0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.75, 0.25], [0.8, 0.2]]
                    else:
                        weights_choices = [[0.4, 0.4, 0.2], [0.4, 0.3, 0.3], [0.5, 0.25, 0.25], [0.6, 0.2, 0.2], [0.8, 0.1, 0.1]]
                else:
                    weights_choices = [None]
                for weights in weights_choices:
                    annualized_return, sharpe_ratio = trade.dca_trade(
                        etfs=etfs,
                        freq=freq,
                        shares=shares,
                        trading_day=trading_day,
                        rebalance_flag=rebalance_flag,
                        weights=weights
                    )
                    s = pd.Series(
                        data=[str(etfs), freq, str(shares), trading_day, rebalance_flag, str(weights), annualized_return, sharpe_ratio],
                        index=['etfs', 'freq', 'shares', 'trading_day', 'rebalance_flag', 'weights', 'annualized_return', 'sharpe_ratio']
                    )
                    ss.append(s)
dca_share_df = pd.DataFrame(ss)
display(dca_share_df)   

In [0]:
ss = []
for etfs in etfs_choices:
    amt = 1000
    for freq in freq_choices:
        # config trading_day
        if freq == 'weekly':
            trading_day_choices = ['MON', 'TUE', 'WED', 'THU', 'FRI']
        else:
            trading_day_choices = ['START', 'END']
        for trading_day in trading_day_choices:
            if len(etfs) > 1:
                rebalance_flag_choices = [True, False]
            else:
                rebalance_flag_choices = [False]
            for rebalance_flag in rebalance_flag_choices:
                if len(etfs) == 1:
                    weights_choices = [[1]]
                elif len(etfs) == 2:
                    weights_choices = [[0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.75, 0.25], [0.8, 0.2]]
                else:
                    weights_choices = [[0.4, 0.4, 0.2], [0.4, 0.3, 0.3], [0.5, 0.25, 0.25], [0.6, 0.2, 0.2], [0.8, 0.1, 0.1]]
                for weights in weights_choices:
                    shares = None
                    annualized_return, sharpe_ratio = trade.dca_trade(
                        etfs=etfs,
                        freq=freq,
                        shares=shares,
                        amt=amt,
                        trading_day=trading_day,
                        rebalance_flag=rebalance_flag,
                        weights=weights
                    )
                    s = pd.Series(
                        data=[str(etfs), freq, str(shares), trading_day, rebalance_flag, str(weights), annualized_return, sharpe_ratio],
                        index=['etfs', 'freq', 'shares', 'trading_day', 'rebalance_flag', 'weights', 'annualized_return', 'sharpe_ratio']
                    )
                    ss.append(s)
dca_amt_df = pd.DataFrame(ss)
display(dca_amt_df)

#### Rule

In [0]:
import itertools
share_dict = {
    'VGT': 2,
    'VOO': 2,
    'SCHD': 10
}
ss = []
for etfs in etfs_choices:
    shares = [share_dict[etf] for etf in etfs]
    threshold_cand = [-0.02, -0.015, -0.01, -0.005, 0.005, 0.01, 0.015, 0.02]
    for thresholds in itertools.product(*([threshold_cand]*len(etfs))):
        if len(etfs) > 1:
            rebalance_flag_choices = [True, False]
        else:
            rebalance_flag_choices = [False]
        for rebalance_flag in rebalance_flag_choices:
            if len(etfs) == 1:
                weights_choices = [[1]]
            elif len(etfs) == 2:
                weights_choices = [[0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.75, 0.25], [0.8, 0.2]]
            else:
                weights_choices = [[0.4, 0.4, 0.2], [0.4, 0.3, 0.3], [0.5, 0.25, 0.25], [0.6, 0.2, 0.2], [0.8, 0.1, 0.1]]
            for weights in weights_choices:
                annualized_return, sharpe_ratio = trade.rule_trade(
                    etfs=etfs,
                    shares=shares,
                    thresholds=thresholds,
                    rebalance_flag=rebalance_flag,
                    weights=weights
                )
                s = pd.Series(
                        data=[str(etfs), str(thresholds), str(shares), rebalance_flag, str(weights), annualized_return, sharpe_ratio],
                        index=['etfs', 'thresholds', 'shares', 'rebalance_flag', 'weights', 'annualized_return', 'sharpe_ratio']
                )
                print(s)
                ss.append(s)
rule_df = pd.DataFrame(ss)
display(rule_df)

In [0]:
df = pd.concat([dca_share_df, dca_amt_df, rule_df], ignore_index=True)
df = df.sort_values(by=['sharpe_ratio', 'annualized_return'], ascending=False)
display(df)